In [1]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as etree
from datetime import datetime
from datetime import timedelta
from sklearn.linear_model import LinearRegression

## Build risk free

In [2]:
df_rf = pd.DataFrame()
for i in range(2011, 2017):
    tmp = pd.read_csv(f'{i}.csv')[['date', 'yearly_return']]
    tmp['date'] = tmp['date'].apply(lambda x: x[:-9])
    tmp['yearly_return'] = tmp['yearly_return'].apply(lambda x: (1 + x/100)**(1 / tmp.shape[0]) - 1)
    df_rf = pd.concat([df_rf, tmp])

In [3]:
df_rf.shape

(1501, 2)

## Build market return and risk free

In [4]:
sp500 = pd.read_csv('sp500.csv')
sp500['date'] = sp500['Date']
df_rm_rf = sp500.drop('Date', axis=1).merge(df_rf, on='date', how='left')
df_rm_rf['rf'] = df_rm_rf['yearly_return']
df_rm_rf['rm'] = sp500['Adj Close'].pct_change()
df_rm_rf = df_rm_rf[['date', 'rm', 'rf']]
print(df_rm_rf.shape)
df_rm_rf.head()

(1510, 3)


,date,rm,rf
0,2011-01-03,NaN,0.000012
1,2011-01-04,-0.001313,0.000011
2,2011-01-05,0.005007,0.000012
3,2011-01-06,-0.002123,0.000012
4,2011-01-07,-0.001845,0.000012


In [5]:
df_rm_rf_missing = df_rm_rf[df_rm_rf.isna().sum(axis=1) == 1]['date'][1:].values

In [6]:
down_days, up_days = [], []
for i in df_rm_rf_missing:
    i_a_1 = datetime.strptime(i, '%Y-%m-%d') + timedelta(days=1)
    i_a_1 = i_a_1.strftime('%Y-%m-%d')
    i_m_1 = datetime.strptime(i, '%Y-%m-%d') - timedelta(days=1)
    i_m_1 = i_m_1.strftime('%Y-%m-%d')
    while sum(df_rf['date'] == i_a_1) == 0:
        i_a_1 = datetime.strptime(i_a_1, '%Y-%m-%d') + timedelta(days=1)
        i_a_1 = i_a_1.strftime('%Y-%m-%d')
    down_days.append(i_a_1)
    while sum(df_rf['date'] == i_m_1) == 0:
        i_m_1 = datetime.strptime(i_m_1, '%Y-%m-%d') - timedelta(days=1)
        i_m_1 = i_m_1.strftime('%Y-%m-%d')
    up_days.append(i_m_1)

In [7]:
for idx, day in enumerate(df_rm_rf_missing):
    tmp_rf_down = df_rf[df_rf['date'] == down_days[idx]]['yearly_return'].values[0]
    tmp_rf_up = df_rf[df_rf['date'] == up_days[idx]]['yearly_return'].values[0]
    tmp_rf = 0.5 * (tmp_rf_down + tmp_rf_up)
    df_rm_rf.iloc[df_rm_rf[df_rm_rf['date'] == day].index, 2] = tmp_rf

In [258]:
df_rm_rf.to_csv('rm_rf.csv', index=False)

## Build return as target

### load final

In [47]:
df = pd.read_csv('viable_final2.csv').drop('Unnamed: 0', axis=1)
print(df.columns)
print(df.shape)
df.head()

Index(['Ticker Symbol', 'Period Ending', 'Accounts Payable',
       'Accounts Receivable', 'Add'l income/expense items', 'After Tax ROE',
       'Capital Expenditures', 'Capital Surplus', 'Cash Ratio',
       'Cash and Cash Equivalents', 'Changes in Inventories', 'Common Stocks',
       'Cost of Revenue', 'Current Ratio', 'Deferred Asset Charges',
       'Deferred Liability Charges', 'Depreciation',
       'Earnings Before Interest and Tax', 'Earnings Before Tax',
       'Effect of Exchange Rate',
       'Equity Earnings/Loss Unconsolidated Subsidiary', 'Fixed Assets',
       'Goodwill', 'Gross Margin', 'Gross Profit', 'Income Tax',
       'Intangible Assets', 'Interest Expense', 'Inventory', 'Investments',
       'Liabilities', 'Long-Term Debt', 'Long-Term Investments',
       'Minority Interest', 'Misc. Stocks', 'Net Borrowings', 'Net Cash Flow',
       'Net Cash Flow-Operating', 'Net Cash Flows-Financing',
       'Net Cash Flows-Investing', 'Net Income', 'Net Income Adjustments',
  

,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,Cash and Cash Equivalents,...,symbol,open,close,low,high,volume,Last PE,Last PE2,Next PE,Next PE2
0,AAL,2012-12-31,3.068000e+09,-222000000.0,-1.961000e+09,23.0,-1.888000e+09,4.695000e+09,53.0,1.330000e+09,...,AAL,12.850000,13.500000,12.840000,13.680000,7005600.0,2011-12-26,2011-12-19,2013-07-08,2013-07-15
1,AAL,2013-12-31,4.975000e+09,-93000000.0,-2.723000e+09,67.0,-3.114000e+09,1.059200e+10,75.0,2.175000e+09,...,AAL,24.740000,25.250000,24.629999,25.250000,7166600.0,2012-12-25,2012-12-18,2014-07-08,2014-07-15
2,AAL,2014-12-31,4.668000e+09,-160000000.0,-1.500000e+08,143.0,-5.311000e+09,1.513500e+10,60.0,1.768000e+09,...,AAL,53.900002,53.630001,53.320000,54.639999,10626000.0,2013-12-25,2013-12-18,2015-07-08,2015-07-15
3,AAL,2015-12-31,5.102000e+09,352000000.0,-7.080000e+08,135.0,-6.151000e+09,1.159100e+10,51.0,1.085000e+09,...,AAL,42.540001,42.349998,41.830002,42.570000,6788900.0,2014-12-25,2014-12-18,2016-07-07,2016-07-14
4,AAP,2012-12-28,2.409453e+09,-89482000.0,6.000000e+05,32.0,-2.711820e+08,5.202150e+08,23.0,5.981110e+08,...,AAP,70.919998,71.510002,70.790001,71.849998,422100.0,2011-12-23,2011-12-16,2013-07-05,2013-07-12


### load adjusted price

In [68]:
df_price = pd.read_csv('prices-split-adjusted.csv')
df_price['date'] = df_price['date'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y').strftime('%Y-%m-%d'))
print(df_price.columns)
print(df_price.shape)
df_price.head()

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')
(851264, 7)


,date,symbol,open,close,low,high,volume
0,2010-01-04,A,22.453504,22.389128,22.267525,22.625180,3815500
1,2010-01-05,A,22.324749,22.145923,22.002861,22.331903,4186000
2,2010-01-06,A,22.067240,22.067240,22.002861,22.174536,3243700
3,2010-01-07,A,22.017168,22.038626,21.816881,22.045780,3095100
4,2010-01-08,A,21.917024,22.031474,21.745350,22.067240,3733900


In [10]:
companies = df['Ticker Symbol'].unique()
df_lite = df[['Ticker Symbol', 'Period Ending']]

### three month return

In [48]:
# dict to save data
three_m_rtn = {'Ticker Symbol':[], 'date': [], 'three_m_rtn': []}
for company in companies:
    endings = df_lite[df_lite['Ticker Symbol'] == company]['Period Ending'].values
    for ending in endings:
        b_idx = df_price[(df_price['date'] == ending) & (df_price['symbol'] == company)].index
        e_idx = b_idx + 63
        if_in = df_price.iloc[e_idx, ]['symbol'] == company
        if if_in.bool():
            e_price = df_price.iloc[e_idx, 3].values
            b_price = df_price.iloc[b_idx, 3].values
            tmp_rtn = (e_price - b_price) / b_price
            three_m_rtn['Ticker Symbol'].append(company)
            three_m_rtn['date'].append(ending)
            three_m_rtn['three_m_rtn'].append(tmp_rtn[0])
        else:
            # log errors
            print(company, ending)

In [52]:
df_three_m_rtn = pd.DataFrame(three_m_rtn)

In [102]:
print(df_three_m_rtn.shape)
df_three_m_rtn.head()

(1474, 3)


,Ticker Symbol,date,three_m_rtn
0,AAL,2012-12-31,0.150370
1,AAL,2013-12-31,0.493465
2,AAL,2014-12-31,-0.082976
3,AAL,2015-12-31,-0.070366
4,AAP,2012-12-28,0.158300


## Calculate VaR

In [123]:
VaR = {'Ticker Symbol':[], 'date': [], 'VaR': []}
for company in companies:
    # get endings for each company
    endings = df_lite[df_lite['Ticker Symbol'] == company]['Period Ending'].values
    for ending in endings:
        # idx on ending date
        b_idx = df_price_rm_rf[(df_price_rm_rf['Ticker Symbol'] == company) & 
                               (df_price_rm_rf['date'] == ending)].index[0]
        # idx for half a year ago
        e_idx = b_idx + 126
        # if company matches
        if_in = df_price_rm_rf.iloc[e_idx, ]['Ticker Symbol'] == company
        if if_in:
            # get y using idx
            rtns = df_price_rm_rf.iloc[b_idx:e_idx, ].apply(lambda row: row['rtn'] - row['rf'], axis=1)
            # calculate VaR
            tmp_VaR = np.percentile(rtns, 5)
            # save company, ending date and beta
            VaR['Ticker Symbol'].append(company)
            VaR['date'].append(ending)
            VaR['VaR'].append(tmp_VaR)
        else:
            # log errors
            print(company, ending)

In [124]:
df_VaR = pd.DataFrame(VaR)
print(df_VaR.shape)
df_VaR.head()

(1474, 3)


,Ticker Symbol,date,VaR
0,AAL,2012-12-31,-0.034344
1,AAL,2013-12-31,-0.029382
2,AAL,2014-12-31,-0.043457
3,AAL,2015-12-31,-0.044833
4,AAP,2012-12-28,-0.017931


## Calculate $\beta$

### calculate daily return

In [69]:
# calculate daily return
df_price['rtn'] = df_price.groupby(['symbol'])['close'].pct_change()
df_price = df_price.dropna()

### calculate CAPM

In [75]:
# merge stock return with rt and rf
df_price_rm_rf = df_rm_rf.drop(0).merge(df_price, on='date', how='left')

In [76]:
# change column name
df_price_rm_rf['Ticker Symbol'] = df_price_rm_rf['symbol']
df_price_rm_rf.drop('symbol', axis=1, inplace=True)

In [77]:
# sort by symbo and date so I can use index later
df_price_rm_rf = df_price_rm_rf.sort_values(['Ticker Symbol', 'date']).reset_index(drop=True)

In [78]:
def get_exp_weights(halftime, trail=126):
    lamb = 2**(-1/halftime)
    wei = [lamb**i for i in range(1,trail+1)]
    return wei

In [79]:
# save beta in dict for merging with main dataset
alpha_beta_res = {'Ticker Symbol':[], 'date': [], 
                  'beta': [], 'exp_weighted_alpha': [], 'std_res': []}
weights = get_exp_weights(42)
for company in companies:
    # get endings for each company
    endings = df_lite[df_lite['Ticker Symbol'] == company]['Period Ending'].values
    for ending in endings:
        # idx on ending date
        e_idx = df_price_rm_rf[(df_price_rm_rf['Ticker Symbol'] == company) & 
                               (df_price_rm_rf['date'] == ending)].index[0]
        # idx for half a year ago
        b_idx = e_idx - 126
        # if company matches
        if_in = df_price_rm_rf.iloc[b_idx, ]['Ticker Symbol'] == company
        if if_in:
            # get X using idx
            X = df_price_rm_rf.iloc[b_idx:e_idx, ].apply(lambda row: row['rm'] - row['rf'], axis=1)
            # get y using idx
            y = df_price_rm_rf.iloc[b_idx:e_idx, ].apply(lambda row: row['rtn'] - row['rf'], axis=1)
            # CAPM
            m = np.shape(X)[0]
            X_full_T = np.matrix([np.ones(m), X]).T
            y_T = np.matrix(y).T
            p_mat = np.linalg.inv(X_full_T.T.dot(X_full_T)).dot(X_full_T.T).dot(y_T)
            # calculate exp weighted alpha
            aplha = y - p_mat[1, 0] * X
            aplha_exp_weight_sum = sum(aplha * weights)
            # calculate sd of res
            res = aplha - p_mat[0, 0]
            res_std = np.std(res)
            # save company, ending date and beta
            alpha_beta_res['Ticker Symbol'].append(company)
            alpha_beta_res['date'].append(ending)
            alpha_beta_res['beta'].append(p_mat[1, 0])
            alpha_beta_res['exp_weighted_alpha'].append(aplha_exp_weight_sum)
            alpha_beta_res['std_res'].append(res_std)
        else:
            # log errors
            print(company, ending)

In [80]:
df_alpha_beta_res = pd.DataFrame(alpha_beta_res)

In [107]:
print(df_alpha_beta_res.shape)
df_alpha_beta_res.head()

(1474, 5)


,Ticker Symbol,date,beta,exp_weighted_alpha,std_res
0,AAL,2012-12-31,0.357608,-0.099730,0.028314
1,AAL,2013-12-31,1.124389,0.093856,0.022369
2,AAL,2014-12-31,1.661482,-0.055035,0.023906
3,AAL,2015-12-31,1.033788,0.097942,0.018811
4,AAP,2012-12-28,0.601798,0.042358,0.018394


## Merge data

In [114]:
df = pd.read_csv('viable_final2_target_acquired.csv')

In [115]:
df_merged = df.merge(df_alpha_beta_res, how='left', on=['date', 'Ticker Symbol'])

In [116]:
df_merged = df_merged.merge(df_three_m_rtn, how='left', on=['date', 'Ticker Symbol'])

In [117]:
df_merged = df_merged.merge(df_VaR, how='left', on=['date', 'Ticker Symbol'])

In [120]:
df_merged.head()

,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,Cash and Cash Equivalents,...,Last PE,Last PE2,Next PE,Next PE2,half_year_return,beta,exp_weighted_alpha,std_res,three_m_rtn,VaR
0,AAL,2012-12-31,3.068000e+09,-222000000.0,-1.961000e+09,23.0,-1.888000e+09,4.695000e+09,53.0,1.330000e+09,...,2011-12-26,2011-12-19,2013-07-08,2013-07-15,0.217037,0.357608,-0.099730,0.028314,0.150370,-0.046172
1,AAL,2013-12-31,4.975000e+09,-93000000.0,-2.723000e+09,67.0,-3.114000e+09,1.059200e+10,75.0,2.175000e+09,...,2012-12-25,2012-12-18,2014-07-08,2014-07-15,0.661386,1.124389,0.093856,0.022369,0.493465,-0.024731
2,AAL,2014-12-31,4.668000e+09,-160000000.0,-1.500000e+08,143.0,-5.311000e+09,1.513500e+10,60.0,1.768000e+09,...,2013-12-25,2013-12-18,2015-07-08,2015-07-15,-0.272236,1.661482,-0.055035,0.023906,-0.082976,-0.037124
3,AAL,2015-12-31,5.102000e+09,352000000.0,-7.080000e+08,135.0,-6.151000e+09,1.159100e+10,51.0,1.085000e+09,...,2014-12-25,2014-12-18,2016-07-07,2016-07-14,-0.307438,1.033788,0.097942,0.018811,-0.070366,-0.035080
4,AAP,2012-12-28,2.409453e+09,-89482000.0,6.000000e+05,32.0,-2.711820e+08,5.202150e+08,23.0,5.981110e+08,...,2011-12-23,2011-12-16,2013-07-05,2013-07-12,0.145154,0.601798,0.042358,0.018394,0.158300,-0.019798


In [118]:
df_merged.to_csv('viable_final2_MK4.csv', index=False)

In [48]:
df = pd.read_csv('price_adj_truncted.csv')
df['year'] = df['date'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y').year)
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))

In [49]:
min(df['date']), max(df['date'])

(Timestamp('2010-01-04 00:00:00'), Timestamp('2016-12-30 00:00:00'))

In [60]:
len(df[df['date'] == max(df['date'])]['symbol'].unique())

424

In [63]:
len(df[df['year'] == 2016]['symbol'].unique())

424

In [67]:
year_dic = {year: 0 for year in range(2010, 2017)}
for year in range(2010, 2017):
    year_dic[year] = len(df[df['year'] == year]['symbol'].unique())
year_dic

{2010: 407, 2011: 407, 2012: 407, 2013: 423, 2014: 423, 2015: 424, 2016: 424}

In [68]:
price_train = df[df['year'] == 2015]
price_train.to_csv('price_train.csv', index=False)

In [70]:
price_test = df[df['year'] == 2016]
price_test.to_csv('price_test.csv', index=False)

In [4]:
price_train = pd.read_csv('price_train.csv')
price_train.groupby('symbol')['date'].count() == 252

symbol
AAL     True
AAP     True
AAPL    True
ABBV    True
ABC     True
ABT     True
ADBE    True
ADI     True
ADM     True
ADS     True
ADSK    True
AEE     True
AEP     True
AFL     True
AIG     True
AIV     True
AIZ     True
AJG     True
AKAM    True
ALB     True
ALK     True
ALL     True
ALLE    True
ALXN    True
AMAT    True
AME     True
AMG     True
AMGN    True
AMP     True
AMT     True
        ... 
VRSN    True
VRTX    True
VTR     True
VZ      True
WAT     True
WDC     True
WEC     True
WFC     True
WFM     True
WHR     True
WM      True
WMB     True
WMT     True
WU      True
WY      True
WYN     True
WYNN    True
XEC     True
XEL     True
XL      True
XLNX    True
XOM     True
XRAY    True
XRX     True
XYL     True
YHOO    True
YUM     True
ZBH     True
ZION    True
ZTS     True
Name: date, Length: 424, dtype: bool